In [1]:
# Dependencies
import pandas as pd
import numpy as np
import os

In [2]:
# Path of the CSV file
csvfile = "customers.csv"


In [3]:
# Read CSV file into a pandas DataFrame
df = pd.read_csv(csvfile, dtype=object)

In [4]:
# Preview DataFrame
# Note that some rows are missing gender
df.head()

,id,first_name,last_name,email,gender
0,1,David,Jordan,djordan0@home.pl,Male
1,2,Stephen,Riley,sriley1@hugedomains.com,Male
2,3,Evelyn,Grant,egrant2@livejournal.com,Female
3,4,Joe,Mendoza,jmendoza3@un.org,Male
4,5,Benjamin,Rodriguez,brodriguez4@elpais.com,Male


In [5]:
# Use `dropna` to drop any rows where there is missing data
# Notice that index has not been reset and contains a gap (0,1,2,6,7)
df = df.dropna(axis=0)
df.head()

,id,first_name,last_name,email,gender
0,1,David,Jordan,djordan0@home.pl,Male
1,2,Stephen,Riley,sriley1@hugedomains.com,Male
2,3,Evelyn,Grant,egrant2@livejournal.com,Female
3,4,Joe,Mendoza,jmendoza3@un.org,Male
4,5,Benjamin,Rodriguez,brodriguez4@elpais.com,Male


In [6]:
# Drop the 'email' column and reset the index
# Note that after resetting the index, the gap is gone
df = df.drop(['email'], axis=1).reset_index(drop=True)
df.head()

,id,first_name,last_name,gender
0,1,David,Jordan,Male
1,2,Stephen,Riley,Male
2,3,Evelyn,Grant,Female
3,4,Joe,Mendoza,Male
4,5,Benjamin,Rodriguez,Male


In [7]:
# Save the cleaned data to a file called `customers_cleaned.csv`
new_csv = "customers_cleaned.csv"
df.to_csv(new_csv, index=False)


In [8]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [9]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///customers.sqlite")

In [10]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [11]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'

    id = Column(Integer, primary_key=True)
    first_name = Column(Text)
    last_name = Column(Text)
    email = Column(Text)
    gender = Column(Text)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
# More on __repr__: https://stackoverflow.com/questions/1984162/purpose-of-pythons-repr    

In [12]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [13]:
# Load the cleaned csv file into a pandas dataframe
new_df = pd.read_csv(new_csv)


In [14]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = new_df.to_dict(orient='records')

In [15]:
print(data)

[{'id': 1, 'first_name': 'David', 'last_name': 'Jordan', 'gender': 'Male'}, {'id': 2, 'first_name': 'Stephen', 'last_name': 'Riley', 'gender': 'Male'}, {'id': 3, 'first_name': 'Evelyn', 'last_name': 'Grant', 'gender': 'Female'}, {'id': 4, 'first_name': 'Joe', 'last_name': 'Mendoza', 'gender': 'Male'}, {'id': 5, 'first_name': 'Benjamin', 'last_name': 'Rodriguez', 'gender': 'Male'}, {'id': 6, 'first_name': 'Kathleen', 'last_name': 'Crawford', 'gender': 'Female'}, {'id': 7, 'first_name': 'Daniel', 'last_name': 'Marshall', 'gender': 'Male'}, {'id': 8, 'first_name': 'Diana', 'last_name': 'Coleman', 'gender': 'Female'}, {'id': 9, 'first_name': 'Frances', 'last_name': 'Black', 'gender': 'Female'}, {'id': 10, 'first_name': 'Tammy', 'last_name': 'Arnold', 'gender': 'Female'}, {'id': 11, 'first_name': 'Jeffrey', 'last_name': 'Matthews', 'gender': 'Male'}, {'id': 12, 'first_name': 'Roy', 'last_name': 'Harrison', 'gender': 'Male'}, {'id': 13, 'first_name': 'Ernest', 'last_name': 'Bishop', 'gender'

In [16]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [17]:
# Save the reference to the `customers` table as a variable called `table`
table = sqlalchemy.Table('customers', metadata, autoload=True)

In [18]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [19]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [20]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from customers limit 5").fetchall()

[(1, 'David', 'Jordan', None, 'Male'),
 (2, 'Stephen', 'Riley', None, 'Male'),
 (3, 'Evelyn', 'Grant', None, 'Female'),
 (4, 'Joe', 'Mendoza', None, 'Male'),
 (5, 'Benjamin', 'Rodriguez', None, 'Male')]